# Valle del Cauca Local Commerce - Analysis

In [1]:
import os
import sys
import duckdb

sys.path.append("../ingest")
import config # type: ignore

## Check for Files in Clean Data Directory

In [2]:
clean_data = "." + config.Local_Dir.Local_Commerce["clean"]
print(os.listdir(clean_data))

['clean_localcommerce.parquet']


## Duckdb Relation

In [3]:
conn = duckdb.connect()
rel = conn.from_parquet(clean_data + config.datasets["local-commerce"]["filename-clean"])

## Dataset Summary

In [4]:
summary = rel.query(virtual_table_name="summary", sql_query="SUMMARIZE rel")
summary

┌─────────────┬─────────────┬─────────┬──────────────┬───────────────┬────────────────────┬────────────────────┬────────────────────┬───────────────────┬────────────────────┬───────┬─────────────────┐
│ column_name │ column_type │   min   │     max      │ approx_unique │        avg         │        std         │        q25         │        q50        │        q75         │ count │ null_percentage │
│   varchar   │   varchar   │ varchar │   varchar    │     int64     │      varchar       │      varchar       │      varchar       │      varchar      │      varchar       │ int64 │  decimal(9,2)   │
├─────────────┼─────────────┼─────────┼──────────────┼───────────────┼────────────────────┼────────────────────┼────────────────────┼───────────────────┼────────────────────┼───────┼─────────────────┤
│ DEPTO       │ UTINYINT    │ 1       │ 10           │            11 │ 4.524233296823659  │ 3.3310879828536777 │ 2                  │ 3                 │ 7                  │ 14608 │            0.

## Top Departments by Local Commerce Value (COP)

- Antioquia = 1.
- Atlantico = 2.
- Bogota = 3.
- Bolivar = 4.
- Cundinamarca = 5.
- Risaralda = 6.
- Santander = 7.
- Tolima = 8.
- Valle del Cauca = 9.

In [5]:
top_dptos = rel.query(
    virtual_table_name="top_dptos",
    sql_query="""SELECT DEPTO, SUM(VENTA) TOTAL_VENTA
    FROM rel
    WHERE DEPTO <> 10
    GROUP BY DEPTO
    ORDER BY TOTAL_VENTA desc
    LIMIT 10;"""
)
top_dptos

┌───────┬────────────────┐
│ DEPTO │  TOTAL_VENTA   │
│ uint8 │     double     │
├───────┼────────────────┤
│     1 │ 163292606206.0 │
│     2 │  69129071523.0 │
│     3 │  41760972582.0 │
│     4 │  36942168525.0 │
│     5 │  22118532026.0 │
│     6 │  16087805109.0 │
│     7 │  11434499814.0 │
│     8 │   9693001120.0 │
│     9 │   7362648460.0 │
└───────┴────────────────┘

## Top Valle del Cauca Commerced Goods by Value (COP)

- Comercio al por menor en establecimientos no especializados y especializados en alimentos = **471-472**.
- Comercio al por mayor especializado de otros productos y no especializado = **466-469**.
- Comercio de vehículos automotores = **451**.
- Comercio al por mayor de materias primas agropecuarias, animales vivos, alimentos, bebidas y tabaco: **462-463**.
- Comercio al por menor en establecimientos no especializados, productos diferentes de alimentos = **473**.
- Comercio al por menor de equipos de informática, comunicaciones y otros enseres domésticos en establecimientos especializados = **474-475**.
- Comercio al por mayor de artículos y enseres domésticos (incluidas prendas de vestir), maquinaria y equipo, comercio al por mayor especializado de otros productos y comercio no especializado = **464-465**.
- Comercio de partes, piezas (autopartes) y accesorios (lujos) para vehículos automotores; comercio de motocicletas y de sus partes, piezas y accesorios = **453-454**.
- Comercio al por menor de artículos culturales y de entretenimiento; y de otros productos en establecimientos especializados, incluido textiles = **476-477**.

In [6]:
top_valle_commerce = rel.query(
    virtual_table_name="top_valle_commerce",
    sql_query="""SELECT CORRELA_9, SUM(VENTA) TOTAL_VENTA
    FROM rel
    WHERE DEPTO = 9
    GROUP BY CORRELA_9
    ORDER BY TOTAL_VENTA desc
    LIMIT 10;"""
)
top_valle_commerce

┌───────────┬──────────────┐
│ CORRELA_9 │ TOTAL_VENTA  │
│  varchar  │    double    │
├───────────┼──────────────┤
│ 471-472   │ 1311630433.0 │
│ 466-469   │ 1131737694.0 │
│ 451       │ 1127794343.0 │
│ 462-463   │ 1072320391.0 │
│ 473       │  701005381.0 │
│ 474 - 475 │  648857657.0 │
│ 464-465   │  590153584.0 │
│ 453-454   │  432357041.0 │
│ 476 - 477 │  346791936.0 │
└───────────┴──────────────┘